## 1. Quá trình dự báo trên mô hình GRU

Thêm thư viện

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import GRUV2 as GRU

Nạp dữ liệu từ file csv

In [2]:

start = '01/06/2020'
end = '01/06/2022'
pre_day = 30
stock="xrp"
# df = investpy.get_crypto_historical_data(crypto=stock ,from_date=start, to_date=end)
df=pd.read_csv('xrp_train_9_1.csv')
df = pd.DataFrame(df)
df

,Date,Open,High,Low,Close,Volume,Currency
0,2020-06-01,0.20254,0.21184,0.20132,0.21090,1662996864,USD
1,2020-06-02,0.21090,0.21452,0.19826,0.20317,2046863744,USD
2,2020-06-03,0.20316,0.20475,0.20104,0.20411,1226735360,USD
3,2020-06-04,0.20411,0.20713,0.20121,0.20438,1357705728,USD
4,2020-06-05,0.20438,0.20567,0.20174,0.20282,1165720704,USD
...,...,...,...,...,...,...,...
653,2022-03-16,0.76586,0.79427,0.75816,0.79190,560469888,USD
654,2022-03-17,0.79204,0.80125,0.78330,0.79331,436895776,USD
655,2022-03-18,0.79370,0.80049,0.77704,0.79661,362681824,USD
656,2022-03-19,0.79652,0.82648,0.79181,0.81970,467541792,USD


Chuẩn hóa dữ liệu

In [3]:
df['H-L'] = df['High'] - df['Low']
df['O-C'] = df['Open'] - df['Close']
ma_1 = 7
ma_2 = 14
ma_3 = 21
# Đường Simple Moving Average (SMA)
# là đường trung bình động đơn giản được tính bằng trung bình cộng các mức giá đóng cửa trong một khoảng thời gian giao dịch nhất định.
# Vd MA_7 = Khoảng thời gian (7 ngày về sau tính cả ngày đang tính)
df[f'SMA_{ma_1}'] = df['Open'].rolling(window=ma_1).mean()
df[f'SMA_{ma_2}'] = df['Open'].rolling(window=ma_2).mean()
df[f'SMA_{ma_3}'] = df['Open'].rolling(window=ma_3).mean()

df[f'SD_{ma_1}'] = df['Open'].rolling(window=ma_1).std()
df[f'SD_{ma_3}'] = df['Open'].rolling(window=ma_3).std()
df.dropna(inplace=True)

df.to_csv(f"{stock}_9_1_standardized.csv")
print(df)

           Date     Open     High      Low    Close      Volume Currency  \
20   2020-06-21  0.18793  0.18840  0.18577  0.18606   623938624      USD   
21   2020-06-22  0.18606  0.19018  0.18584  0.18941  1066312000      USD   
22   2020-06-23  0.18942  0.19042  0.18839  0.18886   793852224      USD   
23   2020-06-24  0.18886  0.19010  0.18179  0.18351  1010395648      USD   
24   2020-06-25  0.18352  0.18406  0.17897  0.18155   912398528      USD   
..          ...      ...      ...      ...      ...         ...      ...   
653  2022-03-16  0.76586  0.79427  0.75816  0.79190   560469888      USD   
654  2022-03-17  0.79204  0.80125  0.78330  0.79331   436895776      USD   
655  2022-03-18  0.79370  0.80049  0.77704  0.79661   362681824      USD   
656  2022-03-19  0.79652  0.82648  0.79181  0.81970   467541792      USD   
657  2022-03-20  0.81958  0.82170  0.79651  0.80606   344777408      USD   

         H-L      O-C     SMA_7    SMA_14    SMA_21      SD_7     SD_21  
20   0.00263 

Scale Miền giá trị lại

In [4]:
# Process Data
pre_day = 30
scala_x = MinMaxScaler()
scala_y = MinMaxScaler()
cols_x = ['H-L', 'O-C', f'SMA_{ma_1}', f'SMA_{ma_2}', f'SMA_{ma_3}', f'SD_{ma_1}', f'SD_{ma_3}']
cols_y = ['Open']
scaled_data_x = scala_x.fit_transform(df[cols_x].values.reshape(-1, len(cols_x)))
scaled_data_y = scala_y.fit_transform(df[cols_y].values.reshape(-1, len(cols_y)))

In [5]:
x_total = []
y_total = []

for i in range(pre_day, len(df)):
    x_total.append(scaled_data_x[i-pre_day:i])
    y_total.append(scaled_data_y[i])
x_total=np.array(x_total)
y_total=np.array(y_total)

In [6]:
# Build Model
model = Sequential()

model.add(GRU(units=60, return_sequences=True, input_shape=(x_total.shape[1], x_total.shape[2])))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60))
model.add(Dropout(0.2))
model.add(Dense(units=len(cols_y)))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_total, y_total, epochs=60, steps_per_epoch=40, use_multiprocessing=True)
model.save(f"{stock}91.h5")
print("Done Training Model")

Epoch 1/60
40/40 [==============================] - 11s 27ms/step - loss: 0.0235
Epoch 2/60
40/40 [==============================] - 1s 28ms/step - loss: 0.0108
Epoch 3/60
40/40 [==============================] - 1s 29ms/step - loss: 0.0093
Epoch 4/60
40/40 [==============================] - 1s 26ms/step - loss: 0.0093
Epoch 5/60
40/40 [==============================] - 1s 30ms/step - loss: 0.0076
Epoch 6/60
40/40 [==============================] - 1s 31ms/step - loss: 0.0068
Epoch 7/60
40/40 [==============================] - 1s 31ms/step - loss: 0.0058
Epoch 8/60
40/40 [==============================] - 1s 30ms/step - loss: 0.0049
Epoch 9/60
40/40 [==============================] - 1s 31ms/step - loss: 0.0046
Epoch 10/60
40/40 [==============================] - 1s 31ms/step - loss: 0.0038
Epoch 11/60
40/40 [==============================] - 1s 30ms/step - loss: 0.0046
Epoch 12/60
40/40 [==============================] - 1s 30ms/step - loss: 0.0047
Epoch 13/60
40/40 [=================